In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import sqlite3
from datetime import datetime
import random
from dateutil.relativedelta import relativedelta
import tkinter as tk
from tkinter import messagebox, simpledialog, ttk

# Initialize Mediapipe Pose Model and SQLite Database
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Database connection
conn = sqlite3.connect('gym_management.db')
c = conn.cursor()

# Create customers table if it doesn't exist
c.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT PRIMARY KEY,
    name TEXT,
    age INTEGER,
    contact TEXT,
    address TEXT,
    email TEXT,
    membership_plan TEXT,
    join_date TEXT,
    end_date TEXT,
    fee REAL
)
''')
conn.commit()

# Fee Structure
fee_structure = {
    "monthly": 50.0,
    "quarterly": 135.0,
    "yearly": 500.0
}

# Exercise Types
EXERCISE_TYPES = ["curls", "squats", "shoulder_press"]

# Generate a unique Customer ID
def generate_customer_id():
    return "GYM" + str(random.randint(1000, 9999))

# Add new customer
def add_customer(name, age, contact, address, email, membership_plan):
    customer_id = generate_customer_id()
    join_date = datetime.now()
    end_date = join_date + relativedelta(months={"monthly": 1, "quarterly": 3, "yearly": 12}.get(membership_plan, 0))
    
    join_date_str = join_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    fee = fee_structure.get(membership_plan, 50.0)

    customer_data = (customer_id, name, age, contact, address, email, membership_plan, join_date_str, end_date_str, fee)
    c.execute('INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', customer_data)
    conn.commit()
    return customer_id

# Update customer details
def update_customer(customer_id, name=None, age=None, contact=None, address=None, email=None, membership_plan=None):
    fields_to_update = []
    values = []

    if name: fields_to_update.append("name = ?"), values.append(name)
    if age: fields_to_update.append("age = ?"), values.append(age)
    if contact: fields_to_update.append("contact = ?"), values.append(contact)
    if address: fields_to_update.append("address = ?"), values.append(address)
    if email: fields_to_update.append("email = ?"), values.append(email)
    if membership_plan: fields_to_update.append("membership_plan = ?"), values.append(membership_plan)

    if fields_to_update:
        set_clause = ", ".join(fields_to_update)
        values.append(customer_id)
        c.execute(f"UPDATE customers SET {set_clause} WHERE customer_id = ?", tuple(values))
        conn.commit()

# Delete customer
def delete_customer(customer_id):
    c.execute("DELETE FROM customers WHERE customer_id = ?", (customer_id,))
    conn.commit()

# View customer details
def view_customer(customer_id):
    c.execute("SELECT * FROM customers WHERE customer_id = ?", (customer_id,))
    return c.fetchone()

# Calculate angle between three points
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle

# Exercise Tracker
def exercise_tracker(exercise_type):
    cap = cv2.VideoCapture(0)
    counter, stage = 0, None 
    key_points = {
        "curls": ["LEFT_SHOULDER", "LEFT_ELBOW", "LEFT_WRIST"],
        "squats": ["LEFT_HIP", "LEFT_KNEE", "LEFT_ANKLE"],
        "shoulder_press": ["LEFT_SHOULDER", "LEFT_ELBOW", "LEFT_WRIST"]
    }

    key_point_labels = key_points[ exercise_type]

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convert to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark
                coords = [landmarks[getattr(mp_pose.PoseLandmark, point).value] for point in key_point_labels]
                key_coords = [[coord.x, coord.y] for coord in coords]
                
                # Calculate and display angle
                angle = calculate_angle(key_coords[0], key_coords[1], key_coords[2])
                cv2.putText(image, f'{exercise_type.capitalize()} Angle: {int(angle)}', 
                            tuple(np.multiply(key_coords[1], [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Exercise-specific counter logic
                if exercise_type == "curls":
                    if angle > 160:
                        stage = "down"
                    if angle < 30 and stage == 'down':
                        stage = "up"
                        counter += 1

                if exercise_type == "squats":
                    if angle < 90:
                        stage = "down"
                    if angle > 160 and stage == 'down':
                        stage = "up"
                        counter += 1

                if exercise_type == "shoulder_press":
                    if angle < 70:
                        stage = "down"
                    if angle > 140 and stage == 'down':
                        stage = "up"
                        counter += 1

                print(f"{exercise_type.capitalize()} Count: {counter}")
                
            except Exception as e:
                pass

            # Display counter and stage
            cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
            cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, stage if stage else "", (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Render landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
            
            # Display image
            cv2.imshow(f'{exercise_type.capitalize()} Tracker', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# GUI Application
class GymManagementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("POSE-FIT TRACKER")
        self.root.geometry("600x400")
        self.root.configure(bg="#ADD8E6")

        self.title_label = ttk.Label(self.root, text="POSE-FIT TRACKER", font=("Arial", 24, "bold"), foreground="black", background="#ADD8E6")
        self.title_label.pack(pady=20)

        self.frame = ttk.Frame(self.root, padding="10 ", relief="flat", style="TFrame")
        self.frame.pack(fill="both", padx=10, pady=10)

        self.add_button = ttk.Button(self.frame, text="Add Customer", command=self.add_customer, width=15, style="TButton")
        self.add_button.grid(row=0, column=0, padx=5, pady=5)

        self.update_button = ttk.Button(self.frame 
, text="Update Customer", command=self.update_customer, width=15, style="TButton")
        self.update_button.grid(row=0, column=1, padx=5, pady=5)

        self.view_button = ttk.Button(self.frame, text="View Customer", command=self.view_customer, width=15, style="TButton")
        self.view_button.grid(row=1, column=0, padx=5, pady=5)

        self.delete_button = ttk.Button(self.frame, text="Delete Customer", command=self.delete_customer, width=15, style="TButton")
        self.delete_button.grid(row=1, column=1, padx=5, pady=5)

        self.exercise_button = ttk.Button(self.frame, text="Exercise Tracker", command=self.start_exercise_tracker, width=15, style="TButton")
        self.exercise_button.grid(row=2, column=0, padx=5, pady=5)

    def get_customer_details(self):
        name = simpledialog.askstring("Name", "Enter customer's name:")
        age = simpledialog.askinteger("Age", "Enter customer's age:")
        contact = simpledialog.askstring("Contact", "Enter customer's contact:")
        address = simpledialog.askstring("Address", "Enter customer's address:")
        email = simpledialog.askstring("Email", "Enter customer's email:")
        membership_plan = simpledialog.askstring("Membership Plan", "Enter membership plan (monthly, quarterly, yearly):")
        return name, age, contact, address, email, membership_plan

    def add_customer(self):
        details = self.get_customer_details()
        if all(details):
            customer_id = add_customer(*details)
            messagebox.showinfo("Success", f"Customer added successfully! Customer ID: {customer_id}")
        else:
            messagebox.showerror("Error", "All fields are required!")

    def update_customer(self):
        customer_id = simpledialog.askstring("Customer ID", "Enter the customer ID to update:")
        if customer_id:
            customer = view_customer(customer_id)
            if customer:
                details = self.get_customer_details()
                update_customer(customer_id, *details)
                messagebox.showinfo("Success", "Customer details updated successfully!")

            else:
                messagebox.showerror("Error", "Customer not found.")
        else:
            messagebox.showerror("Error", "Customer ID is required!")

    def view_customer(self):
        customer_id = simpledialog.askstring("Customer ID", "Enter the customer ID to view:")
        if customer_id:
            customer = view_customer(customer_id)
            if customer:
                customer_info = f"Customer ID: {customer[0]}\nName: {customer[1]}\nAge: {customer[2]}\nContact: {customer[3]}\n"
                customer_info += f"Address: {customer[4]}\nEmail: {customer[5]}\nMembership Plan: {customer[6]}\nJoin Date: {customer[7]}\n"
                customer_info += f"End Date: {customer[8]}\nFee: ${customer[9]}"
                messagebox.showinfo("Customer Details", customer_info)
            else:
                messagebox.showerror("Error", "Customer not found!")
        else:
            messagebox.showerror("Error", "Customer ID is required!")

    def delete_customer(self):
        customer_id = simpledialog.askstring("Customer ID", "Enter the customer ID to delete:")
        if customer_id:
            delete_customer(customer_id)
            messagebox.showinfo("Success", "Customer deleted successfully!")
        else:
            messagebox.showerror("Error", "Customer ID is required!")

    def start_exercise_tracker(self):
        exercise_type = simpledialog.askstring("Exercise Type", "Enter exercise type (curls, squats, shoulder_press):")
        if exercise_type in EXERCISE_TYPES:
            exercise_tracker(exercise_type)
        else:
            messagebox.showerror("Error", "Invalid exercise type!")

# Running the application
if __name__ == "__main__":
    root = tk.Tk()
    app = GymManagementApp(root)
    root.mainloop()

# Close the database connection
conn.close()

Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 0
Curls Count: 1
Curls Count: 1
Curls Count: 1
Curls Count: 1
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 2
Curls Count: 3
Curls Count: 3
Curls Coun